<a href="https://colab.research.google.com/github/bojiix/RAM-OSI-BAN-AI/blob/rogoz/pr_sd_ra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
import pandas as pd
import numpy as np
import random
import string

In [145]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SD2021/cardio_train.csv',sep=";")
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


Numarul total de persoane, separate dupa gen.

---

In [ ]:
df['gender'].value_counts()

1    45530
2    24470
Name: gender, dtype: int64

  Pentru a afla daca valoarea 1 din coloana 'gender' reprezinta genul feminin sau masculin vom calcula inaltimea medie a celor doua genuri. De asemenea, vom tine cont de presupunerea ca, in medie, inaltimea barbatilor este mai mare decat cea a femeilor.

In [ ]:
df.groupby('gender')['height'].mean()

gender
1    161.355612
2    169.947895
Name: height, dtype: float64

Am extras submultimea formata din coloanele 'gender' si 'height', unde 'gender' are rol de index si 'height' este valoarea acesteia. Pentru a obtine valoarea medie a inaltimii, am aplicat functia mean() asupra submultimii obtinute.                                  
Din rezultatul obtinut si tinand cont de presupunerea mentionata anterior, putem presupune ca valoarea 1 din coloana 'gender' reprezinta genul feminin, deoarece valoarea media a acesteia este mai mica decat cea a valorii 2.

Numarul total de persoane cu boli cardiace, pe grupe de varsta

---



In [ ]:
df.groupby('gender')['age'].min()/365

gender
1    29.750685
2    29.583562
Name: age, dtype: float64

In [ ]:
df.groupby('gender')['age'].max()/365

gender
1    64.934247
2    64.967123
Name: age, dtype: float64

Deoarece am grupat coloana 'age' dupa gen, am aflat valoarea minima si maxima de varsta. Folosind aceasta informatie, am stabilit ca valoare coloanei 'age' va fi cuprinsa intre 29 si 64 de ani.

In [147]:
def age_func(x):
  if (x < 30.0) & (x >= 20.0):
    return '20s'
  elif (x < 40.0) & (x >= 30.0):
    return '30s'
  elif (x < 50.0) & (x >= 40.0):
    return '40s'
  elif (x < 60.0) & (x >= 50.0):
    return '50s'
  elif (x < 70.0) & (x >= 60.0):
    return '60s'


Am definit functia age_func(), care va ajuta la stabilirea categoriei de varsta. Categoriile de varsta sunt: 
* 20s (varsta cuprinsa intre 20 -> 29 de ani)
* 30s (varsta cuprinsa intre 30 -> 39 de ani)
* 40s (varsta cuprinsa intre 40 -> 49 de ani)
* 50s (varsta cuprinsa intre 50 -> 59 de ani)
* 60s (varsta cuprinsa intre 60 -> 69 de ani)

In [166]:
ages = [] 
#print(len(ages))
for i, j in df['age'].iteritems():
    ages.append(age_func(df['age'][i]/365))
#print(ages)
df['age_cat']=ages

In lista ages am salvat categoriile de varsta pentru fiecare intrare din coloana 'age', dupa care am atribuit valorile din lista noii coloane 'age_cat'.

In [167]:
df.pivot_table('id',index='age_cat',columns='cardio',aggfunc='count')

cardio,0,1
age_cat,,
20s,3.0,NaN
30s,1349.0,432.0
40s,12177.0,7448.0
50s,17186.0,18355.0
60s,4306.0,8744.0


In [ ]:
df.groupby(['age_cat'])['id'].count()

age_cat
20.0        3
30.0     1781
40.0    19625
50.0    35541
60.0    13050
Name: id, dtype: int64

Inceput incercari

In [ ]:
df.groupby('gender').agg({'alco':'sum','smoke':'sum','active':'sum','cardio':'sum'})

,alco,smoke,active,cardio
gender,,,,
1,1161,813,36516,22616
2,2603,5356,19745,12363


In [ ]:
df.groupby('gender').agg({'weight':'min', 'height':'min', 'age':'min'})

,weight,height,age
gender,,,
1,21.0,55,10859
2,10.0,65,10798


In [ ]:
df.groupby('gender').agg({'weight':'max', 'height':'max', 'age':'max'})

,weight,height,age
gender,,,
1,200.0,250,23701
2,200.0,207,23713


In [ ]:
df.groupby('gender').agg({"age":['min','max','mean']})

age                     
          min    max          mean
gender                            
1       10859  23701  19510.124577
2       10798  23713  19392.097875

In [163]:
df.groupby('age_cat')['id']